In [1]:
import os
from Crypto.Protocol.KDF import PBKDF2 
from Crypto.Cipher import AES
import numpy as np
import time

In [2]:
%run ./linux_usb_mount.ipynb

/mnt/usb


In [3]:
%run ./AES_Utils.ipynb

## File IO

In [4]:
# Make File
def mkFile(fileName, content, mode='wb'):
    try:
        with open(fileName, mode) as f:
            f.write(content)
        return True
    except:
        return False

In [5]:
# Read file
def rdFile(fileName, lenth=-1, mode='rb'):
    try:
        with open(fileName, mode) as f:
            if lenth < 0:
                content = f.read()
            else:
                content = f.read(lenth)
        return content
    except:
        return None

## Encrypt/Decrypt File

In [6]:
encrypt_extention = '.ECRYP'

In [7]:
def encryptFile(fileName, key, rename=None, batch_size_log2=24):
    batch_size = int(2**batch_size_log2)
    get_file_size = os.path.getsize(fileName)
    file_size = get_file_size
    if rename is None:
        rename = fileName+encrypt_extention
    try:
        fr = open(fileName, 'rb')
        fw = open(rename, 'wb')
        #fw.write(b'ECRYP')
        #fw.write(bytes([batch_size_log2]))
        
        while file_size>0:
            content = fr.read(batch_size)
            if file_size <= batch_size:
                content = aesEncrypt(content, key, padding=True)
            else:
                content = aesEncrypt(content, key, padding=False)
            fw.write(content)
            file_size = file_size-batch_size
    finally:
        if fr:
            fr.close()
        if fw:
            fw.close()
    
    return get_file_size

In [8]:
def decryptFile(fileName, key, rename=None, batch_size_log2=24):
    if encrypt_extention == fileName[-len(encrypt_extention):]:
        get_file_size = os.path.getsize(fileName)
        file_size = get_file_size
        if rename is None:
            rename = fileName[:-len(encrypt_extention)]
        try:
            fr = open(fileName, 'rb')
            #if fr.read(5) != b'ECRYP':
            #    fr.close()
            #    return False
            #batch_size_log2 = int.from_bytes(fr.read(1), byteorder='big')

            batch_size = 2**batch_size_log2
            fw = open(rename, 'wb')

            while file_size>0:
                content = fr.read(batch_size)
                if file_size <= batch_size:
                    content = aesDecrypt(content, key, padding=True)
                else:
                    content = aesDecrypt(content, key, padding=False)
                fw.write(content)
                file_size = file_size-batch_size
        finally:
            if fr:
                fr.close()
            if fw:
                fw.close()
                
        return True
    else:
        return False

In [9]:
### Test Encrypt/Decrypt File

## Test Encrypt & Decrypt

In [10]:
encrypt_file_source = "Shakespeare_original.txt"
encrypt_file_rename = "Shakespeare_ENDE.txt"

In [11]:
encrypt_file_source = "ubuntu_original.iso"
encrypt_file_rename = "ubuntu_ENDE.iso"

In [12]:
print("Encrypting: %s"%(encrypt_file_source))
time_start = time.time()
encryptFile(os.path.join(usb_path, encrypt_file_source), key, rename = os.path.join(usb_path, encrypt_file_rename+encrypt_extention))
time_stop = time.time()
print("Duration: %f sec"%(time_stop - time_start))
print("Encrypt done!\n")

Encrypting: ubuntu_original.iso
Duration: 236.051194 sec
Encrypt done!



In [13]:
#r = cmd('mv %s %s'%(os.path.join(usb_path, encrypt_file_source+encrypt_extention), os.path.join(usb_path, encrypt_file_rename+encrypt_extention)))

In [14]:
print("Decrypting: %s"%(encrypt_file_rename+encrypt_extention))
time_start = time.time()
decryptFile(os.path.join(usb_path, encrypt_file_rename+encrypt_extention), key)
time_stop = time.time()
print("Duration: %f sec"%(time_stop - time_start))
print("Decrypt done!\n")

Decrypting: ubuntu_ENDE.iso.ECRYP
Duration: 214.336375 sec
Decrypt done!



In [15]:
### Checksum

## Checksum

In [16]:
print(cmd("md5sum %s"%(os.path.join(usb_path, encrypt_file_source))))

29ca9a17aeb497f5dd79f267649aaf71  /mnt/usb/ubuntu_original.iso



In [17]:
print(cmd("md5sum %s"%(os.path.join(usb_path, encrypt_file_rename+encrypt_extention))))

f5416aa419c6ef1941caf0832460a12a  /mnt/usb/ubuntu_ENDE.iso.ECRYP



In [18]:
print(cmd("md5sum %s"%(os.path.join(usb_path, encrypt_file_rename))))

29ca9a17aeb497f5dd79f267649aaf71  /mnt/usb/ubuntu_ENDE.iso



In [19]:
def tree_encrypt(key, targetFolder = r'/mnt/usb/target'):
    for root, dirs, files in os.walk(targetFolder):
        for fileName in files:
            fullFileName = os.path.join(root, fileName)
            
            if encrypt_extention in fullFileName:
                continue
            
            print(fullFileName)
            
            # Encrypt
            #encryptFile(os.path.join(usb_path, fullFileName), key)
            # Delete file
            #os.system('rm -f "%s"'%fullFileName)
            #os.system('del "%s"'%fullFileName)
    